## Which Offers Better Returns: Bitcoin's Volatility or the S&P 500's Consistency?

### Objective



In [28]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [29]:
# Import data and convert 'Date' value
btc_data = pd.read_csv('./Data/btc_data_aligned_with_sp500.csv', encoding='cp1252')
sp500_data = pd.read_csv('./Data/sp500_data_aligned_with_btc_data.csv', encoding='cp1252')

btc_data['Date'] = pd.to_datetime(btc_data['Date'])
sp500_data['Date'] = pd.to_datetime(sp500_data['Date'])

# Rename the 'Close' columns for clarity
btc_data.rename(columns={"Close": "BTC_Price"}, inplace=True)
sp500_data.rename(columns={"Close": "SP500_Price"}, inplace=True)

# Merging datasets
combined_data = pd.merge(btc_data[['Date', 'BTC_Price']], sp500_data[['Date', 'SP500_Price']], on='Date', how='inner')

# Ensure that Date is the index for the dataset
combined_data.set_index('Date', inplace=True)

combined_data

,BTC_Price,SP500_Price
Date,,
2014-09-18,424.440002,2011.359985
2014-09-19,394.795990,2010.400024
2014-09-22,402.152008,1994.290039
2014-09-23,435.790985,1982.770020
2014-09-24,423.204987,1998.300049
...,...,...
2024-04-24,64276.898438,5071.629883
2024-04-25,64481.707031,5048.419922
2024-04-26,63755.320313,5099.959961


### Function for calculate returns:

$$
\text{Returns (\%)} = \left( \frac{\text{Final Price} - \text{Start Price}}{\text{Start Price}} \right) \times 100
$$


In [30]:
# Function to calculate returns for a give frequency
def calculate_returns(prices, freq):
  '''
  Calculate percentage for a give frequency (e.g. 'YE', 'QE', 'M')
  
  Args: 
  prices (Dataframe): A dataframe with columns for a the asset prices.
  freq (str): Frequency ('YE' for yearly, 'QE' for quarterly, 'M' for monthly, 'W' for weekly) 
  '''
  
  return prices.resample(freq).apply(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[0] * 100) # Returns (%) = ((Final Price / Start Price) / Start Price) * 100)

# Filter data from 2017 onwards
filtered_data = combined_data.loc["2017-01-01":] # Starts from 2017-01-01 to the most recent date


### Calculate Annual Returns

In [ ]:
annual_returns = calculate_returns(filtered_data, 'YE')

# Change colmns names for better understanding of the output
annual_returns.rename(columns={"BTC_Price": "BTC_Returns_%", "SP500_Price": "SP500_Returns_%"}, inplace=True)
annual_returns

,BTC_Returns_%,SP500_Price
Date,,
2017-12-31,1304.065822,18.415027
2018-12-31,-75.018852,-7.009394
2019-12-31,82.420803,28.714796
2020-12-31,315.172062,15.292907
2021-12-31,44.834761,28.793058
2022-12-31,-64.263326,-19.953051
2023-12-31,152.396656,24.729487
2024-12-31,32.449927,5.810034
